In [1]:
# from flask_other.app import app
from flask import Flask
from flask_socketio import SocketIO, emit,join_room,leave_room
from threading import Lock
import os,sys
# sys.path.append('/home/kai/data/wei/Chatbot1.0/')
sys.path.append('../../../../../Chatbot1.0/')
# sys.path.append('/home/kai/data/Chatbot1.0/API/v1/frontend/')
app=Flask("demo")
from flask import  render_template,request
# from app.mong_database import MongoManager
import time
import re
# mongo_db=MongoManager(server_ip='chatbotdb')

import urllib.request
from urllib.parse import quote
import pandas as pd
import numpy as np
import time
import re
import requests

from threading import Lock


##################################
url = 'http://localhost:8889/chatbotv1'


async_mode = None

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app, async_mode=async_mode)




thread = None
thread_lock = Lock()
clients = []
@app.route('/')
def index():
    return render_template('index.html', async_mode=socketio.async_mode)


@socketio.on('client_send',namespace='/test')
def client_msg(msg):  
    print(msg['data']) # this the contentext sent by client
    print(type(msg['data']))
    if msg['data'] == "dis":
        print('test123')
        socketio.emit('my_response',{'data':'disconnection, please try again'},room = request.sid, namespace='/test')
        leave_room(request.sid)
#         socket.close()
        disconnnect()
    else:
        response = 'data get'
        socketio.emit('my_response',{'data':response},room = request.sid, namespace='/test')

#sned message to a specific user
def unique_message(uid):
    socketio.emit('unique_messgae',{'data':'are you still here'}, room=uid, namespace='/test')
    
@socketio.on('disconnect',namespace='/test')
def disconnect():
    uid = request.sid
    print(uid + " disconnect")
    socketio.emit()
    clients.remove(uid)
    leave_room(uid)
    last = len(clients)
    socketio.emit
    socketio.emit('unique_message',{'data':'are you still here'}, room = clients[last-1],namespace = '/test')
    
@socketio.on('connect',namespace='/test')
def connect():
    print('connect')
    print(request.sid)
    uid = request.sid
    clients.append(uid)
    print(len(clients))
    
    join_room(uid)
    
    print('above is room')
    socketio.emit('my_response',{'data':'hello'},room = uid,namespace='/test') #the first sentence 




In [ ]:
print('http://0.0.0.0:6006/')
socketio.run(app,'0.0.0.0',6006)

http://0.0.0.0:6006/
connect
e1fb6cec38654707aadb68decbb08ca6
1
above is room
1
<class 'str'>
2
<class 'str'>
3
<class 'str'>
4
<class 'str'>


message handler error
Traceback (most recent call last):
  File "/home/kai/anaconda3/lib/python3.6/site-packages/engineio/server.py", line 411, in _trigger_event
    return self.handlers[event](*args)
  File "/home/kai/anaconda3/lib/python3.6/site-packages/socketio/server.py", line 522, in _handle_eio_message
    self._handle_event(sid, pkt.namespace, pkt.id, pkt.data)
  File "/home/kai/anaconda3/lib/python3.6/site-packages/socketio/server.py", line 458, in _handle_event
    self._handle_event_internal(self, sid, data, namespace, id)
  File "/home/kai/anaconda3/lib/python3.6/site-packages/socketio/server.py", line 461, in _handle_event_internal
    r = server._trigger_event(data[0], namespace, sid, *data[1:])
  File "/home/kai/anaconda3/lib/python3.6/site-packages/socketio/server.py", line 490, in _trigger_event
    return self.handlers[namespace][event](*args)
  File "/home/kai/anaconda3/lib/python3.6/site-packages/flask_socketio/__init__.py", line 251, in _handler
    *args)
  File "

dis
<class 'str'>
test123
e1fb6cec38654707aadb68decbb08ca6 disconnect
e1fb6cec38654707aadb68decbb08ca6 disconnect
